# Get Data from Airtable

This gets the data from the two tables and formats it into dataframes. It's probably not the cleanest thing in the world, but hopefully it'll do for now.

In [1]:
import ast
import pandas as pd
import requests
from graphviz import Source
import textwrap

In [2]:
def mergeD(ugly_data):
    f = ugly_data["fields"]
    i = {"id": ugly_data["id"]}
    f.update(i)
    return f

def get_table(table_name):
    """ """
    auth_headers = {'Authorization': 'Bearer keyYXkjYFw61SeWDk'}
    table_url = 'https://api.airtable.com/v0/appuaXpFiadmP89sq/' + table_name
    r = requests.get(table_url, headers=auth_headers)
    d = eval(ast.literal_eval(str(r.content)[1:])) # this is almost certainly not the right way to do this!
    
    rows = [mergeD(x) for x in d["records"]]
    data_frame = pd.DataFrame(rows)
    return data_frame

In [3]:
project_data = get_table("Techs")
project_data.sample(10)

,Field 1,Images,LongName,Name,Other_people_involved,Owner,Type,id,percentComplete,precursors
83,22,NaN,Learning Experiment,lExp,NaN,NaN,NaN,recsE2Wezl1QpWoL3,0.0,[rec9uCQTZMlQ2BvGe]
82,25,NaN,Path Visualisation,pathVis,NaN,[rec3HypnJiuTkeVBJ],NaN,recru4UXYdMmO7sX8,50.0,NaN
91,6,NaN,Buildings That Are Right All The Time,BTARATA,NaN,NaN,NaN,reczTE03dicIgyCUJ,0.0,[recDfQGNXPAqgNPab]
84,diary,NaN,NaN,NaN,NaN,NaN,NaN,recsL6OYuIKkhHkTD,NaN,NaN
30,pot,NaN,NaN,NaN,NaN,NaN,NaN,recIsrjagKUV7Itx5,NaN,NaN
7,bravo,NaN,NaN,NaN,NaN,NaN,NaN,rec4nY09e0ly4mmOW,NaN,NaN
12,robSense,NaN,NaN,NaN,NaN,NaN,NaN,rec8yWXx9mfOQv0i5,NaN,NaN
67,SLAM,NaN,NaN,NaN,NaN,NaN,NaN,recjNWgcR0K9C0YsJ,NaN,NaN
63,9,NaN,Desk Location,deskLoc,NaN,NaN,NaN,recg3Af1SGeuhckqO,0.0,"[rec3xfI0c7Elj7eJ4, recVqSccDTSaiamg8]"
16,revit2ML,NaN,NaN,NaN,NaN,NaN,NaN,rec9uCQTZMlQ2BvGe,NaN,NaN


In [11]:
node_description_pattern = "{name} [label=\"{longName}\"];"
edge_pattern = "{from_node} -> {to_node};"

node_descriptions = []
edges = []
for i, row in project_data.iterrows():
    try:
#         ln = textwrap.fill(row["LongName"], 21)
        ln = "\n".join( textwrap.wrap(row["LongName"], 21))
    except Exception as e:
        ln = row["LongName"]
    n = node_description_pattern.format(name=row["Name"], longName=ln)
    node_descriptions.append(n)
    if type(row["precursors"]) is str: # nothing reads as NaN, therefore float
        precursors = row["precursors"].split(",")
        for precursor in precursors:
            e = edge_pattern.format(from_node=precursor,
                                    to_node=row["Name"])
            edges.append(e)

g = """digraph G {{
  rankdir  = "LR";
  label    = "Tech Tree of OATMEAL Projects";
  labelloc = "t";
  node [style=filled,
        color="black", 
        fontcolor="white", 
        shape="oval", 
        fixedsize=true, 
        width=2,
        height=1];
  
  {nodes}
  {edges}
  {{rank = same; data; basicRobot; hardware; systemReef; SNAawareness}}
  
}}""".format(edges="\n  ".join(edges), nodes="\n  ".join(node_descriptions))


# print(g)
g.split("\n ")

['digraph G {',
 ' rankdir  = "LR";',
 ' label    = "Tech Tree of OATMEAL Projects";',
 ' labelloc = "t";',
 ' node [style=filled,',
 '       color="black", ',
 '       fontcolor="white", ',
 '       shape="oval", ',
 '       fixedsize=true, ',
 '       width=2,',
 '       height=1];',
 ' ',
 ' potPaper [label="A Plane of Thrones\nPaper"];',
 ' nan [label="nan"];',
 ' robPaper [label="Robotics Papers"];',
 ' nan [label="nan"];',
 ' deskSenseDn [label="Below Desk Sensing"];',
 ' desks [label="Unit Desks"];',
 ' nan [label="nan"];',
 ' nan [label="nan"];',
 ' deskSenseUp [label="Above Desk Sensing"];',
 ' mApp [label="Mobile App"];',
 ' nan [label="nan"];',
 ' nan [label="nan"];',
 ' nan [label="nan"];',
 ' geoEthno2 [label="Geographic/\nEthnographic data 2"];',
 ' nan [label="nan"];',
 ' nan [label="nan"];',
 ' nan [label="nan"];',
 ' nan [label="nan"];',
 ' nan [label="nan"];',
 ' basicBeacon [label="iBeacon Awareness"];',
 ' sensicorn2 [label="Sensicorn 2"];',
 ' geoEthno1 [label="Geo

In [5]:
people_data = get_table("People")

In [6]:
def get_thumb(row):
    try:
        h = row["Headshot"]
        if type(h) is not float:
            # TODO: make this check for small as well
            x = list(h)
            return x[0]['thumbnails']["large"]["url"]
        else:
            return "http://getdrawings.com/images/casper-drawing-8.jpg"
    except Exception as e:
        print(e)
        return "http://getdrawings.com/images/casper-drawing-8.jpg"

people_data["thumb_large"] = people_data.apply(get_thumb, axis=1)
people_data

,Headshot,Initials,Name,Techs,id,thumb_large
0,NaN,KM,Kevin M,[recnXgQGgzh4q3t9M],rec0dpxUhHfgHWU7r,http://getdrawings.com/images/casper-drawing-8...
1,NaN,ARR,Annisa,[recru4UXYdMmO7sX8],rec3HypnJiuTkeVBJ,http://getdrawings.com/images/casper-drawing-8...
2,NaN,BaD,Barry D,NaN,recFKWrjwOUM5oWxI,http://getdrawings.com/images/casper-drawing-8...
3,NaN,RX,AI Richard,NaN,recFUgthTUYQnR7Mz,http://getdrawings.com/images/casper-drawing-8...
4,NaN,NT,NT,"[rec3robMWYUXVp8GD, recli0IRLQZowpWoF, rectkEz...",recHLT6IoCZXwwtp3,http://getdrawings.com/images/casper-drawing-8...
5,NaN,BiD,Bill,"[recmNHHmIUj4nPild, recHpePW60ZlXrLKY]",recHN7P3Wz3YGdAN2,http://getdrawings.com/images/casper-drawing-8...
6,NaN,BC,Barry C,NaN,recMZdSRhVMMJQcOb,http://getdrawings.com/images/casper-drawing-8...
7,"[{'filename': 'MVIMG_20180224_173642.jpg', 'th...",BeD,Ben,"[reccHCPeML8SyRFTp, rec5v7YBAh4XzeOQ7, recSIIp...",rece9qyNSfMqAhZAL,https://dl.airtable.com/nUBlmdvR1iLT0mCbllA1_l...
8,NaN,IV,Ishaan,"[recCdV0IBu62AhiOt, recmjo5fVtiAflmvo, recflCm...",recgcDkgKvrBLH1k0,http://getdrawings.com/images/casper-drawing-8...
9,NaN,AR,Aiden,"[rec5v7YBAh4XzeOQ7, rec2musyOg3qL274O, reczMCu...",recj2rmnndUmxB3Dt,http://getdrawings.com/images/casper-drawing-8...


Aiming for something that looks a bit like this:
![](https://lh3.googleusercontent.com/xm3KUhGDiEDbr5v9jnlTv3kqqBDbTXP497VxE2otSsmHMpdDi5kBZaBnCnWCzknNFZynJw91wm8fRoOS_VqJCuQOA_4dCXYrqkIQG_YHMwmuNsqiPBJnAVRu1e1ZMvywiw9JkQDkCJM=w1430-h1015-no)
So we need these bits of information:

In [7]:
this_project = project_data[project_data.Name == "deskSenseUp"] # "AIlayout" for my face
print(this_project.LongName)
owner = list(this_project.Owner)[0][0]
this_project

8    Above Desk Sensing
Name: LongName, dtype: object


,Field 1,Images,LongName,Name,Other_people_involved,Owner,Type,id,percentComplete,precursors
8,12,NaN,Above Desk Sensing,deskSenseUp,"[recuFyI25ChOwWWhr, rece9qyNSfMqAhZAL]",[recj2rmnndUmxB3Dt],NaN,rec5v7YBAh4XzeOQ7,0.0,"[recDNadVUFPsteXcg, rechRmJoxqqu38jZJ]"


In [8]:
list(people_data[people_data.id == owner].thumb_large)

['http://getdrawings.com/images/casper-drawing-8.jpg']

In [9]:
from IPython.display import Image
from IPython import display

halpers = this_project.Other_people_involved.values[0]
for h in halpers:
    row = people_data[people_data.id == h]
    print(h, row.Name.values[0], row.thumb_large.values[0])
    display.display(row.thumb_large.values[0])
#     Image()

recuFyI25ChOwWWhr Baptiste http://getdrawings.com/images/casper-drawing-8.jpg


'http://getdrawings.com/images/casper-drawing-8.jpg'

rece9qyNSfMqAhZAL Ben https://dl.airtable.com/nUBlmdvR1iLT0mCbllA1_large_MVIMG_20180224_173642.jpg


'https://dl.airtable.com/nUBlmdvR1iLT0mCbllA1_large_MVIMG_20180224_173642.jpg'

In [10]:
#now this is ready to drive the progress ring
list(this_project.percentComplete)[0]#.replace("%", "")

0.0

![](https://dl.airtable.com/vao3Ge65RXuQs17gLU4p_small_MVIMG_20180224_173642.jpg)